# Submit PW calculation
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO calculation

In [3]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

#### Create a diamond cubic crystal structure

In [4]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

<StructureData: uuid: 6919e37a-f710-4f5c-8e38-a7d0f5cf142b (pk: 4936)>

#### Create the _k_-points mesh

In [3]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

<KpointsData: uuid: 64587c03-96d9-466c-a524-968864432003 (pk: 4503)>

#### Define the calculation input parameters

In [4]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP_v0.7_eff_PBE'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### Tell AiiDA which code to use

In [5]:
# Retrieve the code
codename = 'pw-v5.4.0'
code = Code.get_from_string(codename)

In [6]:
# Create a new calculation from the code and set the inputs
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

In [7]:
# Simply store the calculation and submit it
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

Submitted a PwCalculation with pk<4505>


In [12]:
!verdi calculation list

# Last daemon state_updater check: 0h:00m:00s ago (at 14:28:17 on 2017-04-13)
PK    State    Creation    Sched. state    Computer    Type
----  -------  ----------  --------------  ----------  ------

Total results: 0



In [13]:
# Load the calculation node and check if it finished correctly
calculation = load_node(calc.pk)
calculation.has_finished_ok()

True

In [14]:
# We can easily show all the attached outputs
calculation.get_outputs_dict()

{u'output_array': <ArrayData: uuid: 3b32da67-1578-43dd-9e83-3e73b0a0cf28 (pk: 4510)>,
 u'output_array_4510': <ArrayData: uuid: 3b32da67-1578-43dd-9e83-3e73b0a0cf28 (pk: 4510)>,
 u'output_kpoints': <KpointsData: uuid: f155fba8-ccd9-4c55-98c7-9e3347c6e160 (pk: 4508)>,
 u'output_kpoints_4508': <KpointsData: uuid: f155fba8-ccd9-4c55-98c7-9e3347c6e160 (pk: 4508)>,
 u'output_parameters': <ParameterData: uuid: 0f69eb3d-e3d1-4d74-b807-e9b9e7cf35da (pk: 4509)>,
 u'output_parameters_4509': <ParameterData: uuid: 0f69eb3d-e3d1-4d74-b807-e9b9e7cf35da (pk: 4509)>,
 u'remote_folder': <RemoteData: uuid: c1129d33-72d5-470b-91ff-6ac1e8687e54 (pk: 4506)>,
 u'remote_folder_4506': <RemoteData: uuid: c1129d33-72d5-470b-91ff-6ac1e8687e54 (pk: 4506)>,
 u'retrieved': <FolderData: uuid: b536dcd1-86df-4b61-80ce-4062748a0880 (pk: 4507)>,
 u'retrieved_4507': <FolderData: uuid: b536dcd1-86df-4b61-80ce-4062748a0880 (pk: 4507)>}

In [15]:
# There are convenience methods to directly access the results
print "Total energy = {} {}".format(calculation.res.energy, calculation.res.energy_units)

Total energy = -2471.82841453 eV
